<a href="https://colab.research.google.com/github/gjduart/ML-on-Tensorflow/blob/main/RNN_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from keras.preprocessing import sequence

In [ ]:
VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

In [ ]:
(train_data, train_labels),(test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17473536/17464789 [==============================] - 0s 0us/step


In [ ]:
len(train_data[0])

218

In [ ]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(VOCAB_SIZE,32),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,train_labels, epochs=10, validation_split=.2)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


625/625 [==============================] - 74s 116ms/step - loss: 0.4012 - accuracy: 0.8166 - val_loss: 0.3252 - val_accuracy: 0.8640
Epoch 2/10
625/625 [==============================] - 74s 118ms/step - loss: 0.1963 - accuracy: 0.9274 - val_loss: 0.3052 - val_accuracy: 0.8804
Epoch 3/10
625/625 [==============================] - 73s 117ms/step - loss: 0.0942 - accuracy: 0.9697 - val_loss: 0.3733 - val_accuracy: 0.8664
Epoch 4/10
625/625 [==============================] - 77s 123ms/step - loss: 0.0641 - accuracy: 0.9801 - val_loss: 0.4533 - val_accuracy: 0.8640
Epoch 5/10
625/625 [==============================] - 73s 117ms/step - loss: 0.0722 - accuracy: 0.9754 - val_loss: 0.4557 - val_accuracy: 0.8556
Epoch 6/10
625/625 [==============================] - 73s 117ms/step - loss: 0.0317 - accuracy: 0.9910 - val_loss: 0.5274 - val_accuracy: 0.8682
Epoch 7/10
625/625 [==============================] - 72s 116ms/step - loss: 0.0170 - accuracy: 0.9952 - val_loss: 0.5842 - val_accuracy: 0.8

In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
text = 'that movies was just amazing, so amazing'
encoded_text = encode_text(text)

In [ ]:
reverse_word_index = {
    value:key for (key,value) in word_index.items()
}

In [ ]:
def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num!=PAD:
      text+= reverse_word_index[num] + " "

  return text[:-1]

In [ ]:
decode_integers(encoded_text)

'that movies was just amazing so amazing'

In [ ]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

In [ ]:
positive_review = 'The movie was awesome! I really loved it and would watch it again because it was amazingly great'
predict(positive_review)

[0.9379649]


In [ ]:
negative_review = "I hated this movie, I hated it and wouldn't watch it again. Was one of the wrost thing I've ever watched"
predict(negative_review)

[0.02568144]
